In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv("C:/MADS/DATEXP03/Smart Toilet/data/raw/dataset 1.csv")

In [12]:
df

,home,seat,user_id,session,window,urinate,normal,no,hard,soft,weight,heartrate,temperature,comx,comy
0,8868fe4de610d348,d6b7a54f96ce39a4,eefbc3f27093becc,2025-07-27-18-11-28,1,1,1,0,0,0,46.610289,NaN,35.327988,0.029312,0.560280
1,8868fe4de610d348,d6b7a54f96ce39a4,eefbc3f27093becc,2025-07-27-18-11-28,2,1,1,0,0,0,46.573018,67.964602,35.327988,0.029735,0.562222
2,8868fe4de610d348,d6b7a54f96ce39a4,eefbc3f27093becc,2025-07-27-18-11-28,3,1,1,0,0,0,46.528367,64.537815,35.327988,0.029902,0.563786
3,8868fe4de610d348,d6b7a54f96ce39a4,eefbc3f27093becc,2025-07-27-18-11-28,4,1,1,0,0,0,46.425224,62.439024,35.327988,0.031608,0.568907
4,8868fe4de610d348,d6b7a54f96ce39a4,eefbc3f27093becc,2025-07-27-18-11-28,5,1,1,0,0,0,46.325638,60.472441,35.327988,0.033335,0.570972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16383,929b9daaa2c181f9,d6b7a54f96ce39a4,28d3731c392ca13c,2025-07-10-08-08-24,16,0,0,0,0,0,40.344195,NaN,34.722966,0.023062,0.855267
16384,929b9daaa2c181f9,d6b7a54f96ce39a4,28d3731c392ca13c,2025-07-10-08-08-24,17,0,0,0,0,0,NaN,NaN,34.722966,NaN,NaN
16385,929b9daaa2c181f9,d6b7a54f96ce39a4,28d3731c392ca13c,2025-07-10-08-08-24,18,0,0,0,0,0,NaN,NaN,34.722966,NaN,NaN
16386,929b9daaa2c181f9,d6b7a54f96ce39a4,28d3731c392ca13c,2025-07-10-08-08-24,19,0,0,0,0,0,NaN,NaN,34.722966,NaN,NaN


In [9]:
df.shape

(16388, 15)

In [22]:
df.isna().sum().sort_values(ascending=False)

heartrate      4826
weight         2619
comx           2619
comy           2619
temperature     647
seat              0
home              0
normal            0
urinate           0
window            0
session           0
user_id           0
soft              0
no                0
hard              0
dtype: int64

In [21]:
missing_pct = (df.isna().sum() / len(df)) * 100
missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=False)

missing_pct.round(2)

heartrate      29.45
weight         15.98
comx           15.98
comy           15.98
temperature     3.95
dtype: float64

In [23]:
df.duplicated().sum()

np.int64(15)

In [25]:
df[["no","urinate","normal","hard","soft"]].sum(axis=1).value_counts()

1    9280
2    5502
0    1300
3     306
Name: count, dtype: int64

In [26]:
session_buttons = df.groupby("session")[["no","urinate","normal","hard","soft"]].max()

In [27]:
session_buttons["defecation"] = (
    (session_buttons["normal"] == 1) |
    (session_buttons["hard"] == 1) |
    (session_buttons["soft"] == 1)
).astype(int)

In [33]:
# Maar 1 sessie met "Aandrang". Dimensie verwaarlozen
session_buttons[["no"]].value_counts()

no
0     343
1       1
Name: count, dtype: int64

In [35]:
session_buttons["target"] = np.select(
    [
        (session_buttons["urinate"] == 1) & (session_buttons["defecation"] == 0),
        (session_buttons["urinate"] == 0) & (session_buttons["defecation"] == 1),
        (session_buttons["urinate"] == 1) & (session_buttons["defecation"] == 1)
    ],
    [0, 1, 2],
    default=np.nan
)

In [29]:
session_buttons["target"].value_counts()

target
1.0    231
3.0     48
2.0     22
Name: count, dtype: int64